### 原始数据

In [1]:
part_a={
    "date":["2023/11/21","2023/11/30","2023/12/11","2023/12/19","2023/12/28","2024/01/21","2024/01/25","2024/04/08","2024/04/16","2024/04/21","2024/04/28","2024/05/10","2024/05/16","2024/05/22","2024/05/25","2024/05/31","2024/06/02","2024/06/06","2024/06/07","2024/06/20","2024/06/28","2024/07/01","2024/07/07","2024/07/13","2024/07/18","2024/07/23","2024/07/26","2024/07/31","2024/08/08","2024/08/15","2024/08/17","2024/08/28","2024/09/04","2024/09/06","2024/09/07","2024/09/14","2024/09/15","2024/09/21","2024/09/25","2024/09/26","2024/10/07","2024/10/20","2024/10/23","2024/10/27","2024/11/02","2024/11/07","2024/11/09","2024/11/12","2024/11/14","2024/11/16","2024/11/19","2024/11/29","2024/12/10","2024/12/13","2024/12/14","2024/12/19","2024/12/26","2024/12/29","2024/12/31",],
    "numbers":[300,150,100,110,170,109,711,80,70,100,150,200,100,40,55,50,100,50,50,60,13,107,120,70,61,35,10,94,120,120,80,80,94,36,70,88,80,72,100,250,100,100,100,90,50,37,57,61,55,100,45,132,1,180,180,25,123,1,331,]
    }
part_b={
    "date":["2023/11/18","2023/11/28","2023/11/29","2023/11/30","2023/12/5","2023/12/9","2023/12/13","2023/12/18","2023/12/23","2023/12/26","2023/12/27","2024/1/19","2024/1/23","2024/3/16","2024/3/22","2024/4/7","2024/4/8","2024/4/12","2024/4/13","2024/4/17","2024/4/26","2024/4/27","2024/4/28","2024/5/6","2024/5/8","2024/5/9","2024/5/17","2024/5/18","2024/5/21","2024/5/23","2024/5/28","2024/5/30","2024/5/31","2024/6/1","2024/6/6","2024/6/11","2024/6/15","2024/6/18","2024/6/25","2024/7/1","2024/7/5","2024/7/6","2024/7/12","2024/7/16","2024/7/22","2024/7/25","2024/8/1","2024/8/6","2024/8/15","2024/8/16","2024/8/27","2024/9/3","2024/9/4","2024/9/6","2024/9/7","2024/9/13","2024/9/21","2024/9/24","2024/9/25","2024/10/6","2024/10/19","2024/10/21","2024/10/24","2024/10/31","2024/11/6","2024/11/7","2024/11/9","2024/11/13","2024/11/15","2024/11/28","2024/12/9","2024/12/12","2024/12/13","2024/12/17","2024/12/25","2024/12/29","2024/12/30",],
    "numbers":[1550,300,150,200,150,200,50,110,150,170,60,229,2800,100,54,116,110,190,110,350,100,340,29,82,70,200,249,100,40,56,50,50,100,101,55,100,1,60,13,107,120,20,70,61,35,10,94,120,80,120,80,91,4,36,70,168,72,300,50,100,100,100,90,50,39,57,61,155,25,132,1,180,180,25,123,311,20,]
    }

In [2]:
print(part_a)
print(part_b)

{'date': ['2023/11/21', '2023/11/30', '2023/12/11', '2023/12/19', '2023/12/28', '2024/01/21', '2024/01/25', '2024/04/08', '2024/04/16', '2024/04/21', '2024/04/28', '2024/05/10', '2024/05/16', '2024/05/22', '2024/05/25', '2024/05/31', '2024/06/02', '2024/06/06', '2024/06/07', '2024/06/20', '2024/06/28', '2024/07/01', '2024/07/07', '2024/07/13', '2024/07/18', '2024/07/23', '2024/07/26', '2024/07/31', '2024/08/08', '2024/08/15', '2024/08/17', '2024/08/28', '2024/09/04', '2024/09/06', '2024/09/07', '2024/09/14', '2024/09/15', '2024/09/21', '2024/09/25', '2024/09/26', '2024/10/07', '2024/10/20', '2024/10/23', '2024/10/27', '2024/11/02', '2024/11/07', '2024/11/09', '2024/11/12', '2024/11/14', '2024/11/16', '2024/11/19', '2024/11/29', '2024/12/10', '2024/12/13', '2024/12/14', '2024/12/19', '2024/12/26', '2024/12/29', '2024/12/31'], 'numbers': [300, 150, 100, 110, 170, 109, 711, 80, 70, 100, 150, 200, 100, 40, 55, 50, 100, 50, 50, 60, 13, 107, 120, 70, 61, 35, 10, 94, 120, 120, 80, 80, 94, 36,

### 算法部分

编写python函数完成如下功能：

1、输入数据为 part_a、part_b，结构为：

```python
part_a={
    "date":["data1","data2","data3"],
    "numbers":[number1,number2,number3]
    }
part_b={
    "date":["data1","data2","data3"],
    "numbers":[number2,number3，number4]
    }
```

2、求两个数据的最佳匹配，并将匹配按时间顺序输出，匹配规则如下：
- 求 part_a、part_b 中 numbers 完全匹配的个数最多有多少，限制条件为 part_a 中的 number 对应的 date 需要大于等于 part_b 中的 date
- 对于匹配失败的保持原数据不动

3、输出格式如下：
- 输出为表格，有四列（part_a.date, part_a,numbers,part_b,date,part_b.numbers），匹配成功的放在一行
- 匹配失败的按日期要求即 part_a.date > part_b.date，尽量将相近的放在一行
- 输出的表格整体按日期排序

In [3]:
import rich
import pandas as pd
from datetime import datetime


def match_data(part_a, part_b):
    """
    Match data between part_a and part_b based on the following rules:
    - Numbers must match exactly
    - part_a date must be >= part_b date
    - Maximize the number of matches
    - Output a sorted table with matched and unmatched data

    Args:
        part_a: Dictionary with 'date' and 'numbers' lists
        part_b: Dictionary with 'date' and 'numbers' lists

    Returns:
        DataFrame with columns: part_a.date, part_a.numbers, part_b.date, part_b.numbers
    """
    # Convert input data to DataFrames
    df_a = pd.DataFrame(part_a)
    df_b = pd.DataFrame(part_b)

    # Convert date strings to datetime objects for proper comparison
    df_a["date"] = pd.to_datetime(df_a["date"])
    df_b["date"] = pd.to_datetime(df_b["date"])

    # Create a copy of DataFrames to track matched items
    df_a_matched = df_a.copy()
    df_a_matched["matched"] = False
    df_b_matched = df_b.copy()
    df_b_matched["matched"] = False

    # Create an empty DataFrame with proper columns to store matched pairs
    result_columns = ["part_a.date", "part_a.numbers", "part_b.date", "part_b.numbers"]
    result_df = pd.DataFrame(columns=result_columns)

    # Find all potential matches (where numbers match)
    for i, row_a in df_a.iterrows():
        potential_matches = []

        for j, row_b in df_b.iterrows():
            # Check if numbers match and date condition is satisfied
            if row_a["numbers"] == row_b["numbers"] and row_a["date"] >= row_b["date"] and not df_b_matched.loc[j, "matched"]:
                potential_matches.append((j, row_b))

        # If potential matches exist, select the one with the closest date
        if potential_matches:
            potential_matches.sort(key=lambda x: abs((row_a["date"] - x[1]["date"]).total_seconds()))
            best_match_idx, best_match = potential_matches[0]

            # Mark as matched
            df_a_matched.loc[i, "matched"] = True
            df_b_matched.loc[best_match_idx, "matched"] = True

            # Add to matched pairs
            new_row = pd.DataFrame({"part_a.date": [row_a["date"]], "part_a.numbers": [row_a["numbers"]], "part_b.date": [best_match["date"]], "part_b.numbers": [best_match["numbers"]]})
            result_df = pd.concat([result_df, new_row], ignore_index=True)

    # Handle unmatched items
    unmatched_a = df_a_matched[~df_a_matched["matched"]].drop("matched", axis=1)
    unmatched_b = df_b_matched[~df_b_matched["matched"]].drop("matched", axis=1)

    # Add unmatched items from part_a
    for _, row in unmatched_a.iterrows():
        # Try to find closest date in unmatched_b that satisfies date condition
        valid_b = unmatched_b[unmatched_b["date"] <= row["date"]]

        if not valid_b.empty:
            # Find the closest date
            closest_idx = (valid_b["date"] - row["date"]).abs().idxmin()
            closest_b = valid_b.loc[closest_idx]

            new_row = pd.DataFrame({"part_a.date": [row["date"]], "part_a.numbers": [row["numbers"]], "part_b.date": [closest_b["date"]], "part_b.numbers": [closest_b["numbers"]]})
            result_df = pd.concat([result_df, new_row], ignore_index=True)

            # Remove the used row from unmatched_b
            unmatched_b = unmatched_b.drop(closest_idx)
        else:
            # No valid match in part_b
            new_row = pd.DataFrame({"part_a.date": [row["date"]], "part_a.numbers": [row["numbers"]], "part_b.date": [pd.NaT], "part_b.numbers": [None]})  # Use pd.NaT for datetime NA values
            result_df = pd.concat([result_df, new_row], ignore_index=True)

    # Add remaining unmatched items from part_b
    for _, row in unmatched_b.iterrows():
        new_row = pd.DataFrame({"part_a.date": [pd.NaT], "part_a.numbers": [None], "part_b.date": [row["date"]], "part_b.numbers": [row["numbers"]]})  # Use pd.NaT for datetime NA values
        result_df = pd.concat([result_df, new_row], ignore_index=True)

    # Sort the result by part_a date, then part_b date
    result_df = result_df.sort_values(by=["part_a.date", "part_b.date"], na_position="last")

    return result_df


result = match_data(part_a, part_b)

from IPython.display import display, HTML

display(HTML(result.to_html()))

/var/folders/k0/fc50nlsj2r5fgrzbv6736w1h0000gn/T/ipykernel_16596/3007651121.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)


,part_a.date,part_a.numbers,part_b.date,part_b.numbers
44,2023-11-21,300,2023-11-18,1550
0,2023-11-30,150,2023-11-29,150
45,2023-12-11,100,2023-12-09,200
1,2023-12-19,110,2023-12-18,110
2,2023-12-28,170,2023-12-26,170
46,2024-01-21,109,2024-01-19,229
47,2024-01-25,711,2024-01-23,2800
48,2024-04-08,80,2024-04-08,110
49,2024-04-16,70,2024-04-13,110
3,2024-04-21,100,2024-03-16,100
